In [ ]:
# !pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
import os
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import requests
from PIL import Image
from io import BytesIO

In [ ]:
# Create a browser
service = Service()
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
# Create a dir
root_dir = './corpus'
os.makedirs(root_dir, exist_ok=True)
list_of_topics = ["data-speaks", "property", "markets", "companies", "economy", "exchange-rate", "money"]
article_id = 0
img_id = 0

In [ ]:
#Get list of URLs
for topic in tqdm(list_of_topics):
  main_url = f"https://e.vnexpress.net/news/business/{topic}"
  driver.get(main_url)
  time.sleep(1)

  tags_xpath = '//*[@id="vnexpress_folder_list_news"]/div/div[1]/div[1]/a'

  list_of_tags = driver.find_elements(
      By.XPATH, tags_xpath
  )
  list_of_urls = [tag.get_attribute("href") for tag in list_of_tags]

  for url in list_of_urls:
    # Access to url
    try:
      driver.get(url)
      time.sleep(1)
    except:
      raise Exception(f"Cannot access to {url}")

    # Get title
    title_xpath = '//*[@id="detail_page"]/div[1]/div[2]/div/div[2]/div[1]/div/div/h1'
    try:
      title = driver.find_element(
          By.XPATH, title_xpath
      ).text.strip()
    except:
      raise Exception(f"Cannot get title of {url}")

    #  Get author
    author_xpath = '//*[@id="detail_page"]/div[1]/div[2]/div/div[2]/div[2]/a'
    try:
      author = driver.find_element(
          By.XPATH, author_xpath
      ).text.strip()
    except:
      author = '' # cannot set author to None bc of join() method at the end of te code

    # Get thumbnail
    img_xpath = '//*[@id="detail_page"]/div[1]/div[2]/div/div[2]/div[3]/img'
    try:
      img_tags = driver.find_elements(
          By.XPATH, img_xpath
      )
    except:
      raise Exception(f"Cannot get img tags of {url}")
    else:
      img_urls = [img_tag.get_attribute('src') for img_tag in img_tags]
      if img_urls is None:
        img_id += 1
      else:
        for img_url in img_urls:
          img_url_res = requests.get(img_url)
          try:
            img = Image.open(BytesIO(img_url_res.content))
          except:
            raise Exception(f"Cannot open thumbnail of {url}")
          if img.mode == "P":
            img = img.convert("RBG")
          # Save img to .png file
          img_name = f"img_{img_id:05d}.png"
          img_save_path = os.path.join(
              root_dir, img_name
          )
          img.save(img_save_path)
          img_id += 1

    # Get abstract
    abstract_xpath = '//*[@id="detail_page"]/div[1]/div[2]/div/div[2]/span'
    try:
      abstract = driver.find_element(
          By.XPATH, abstract_xpath
      ).text.strip()
    except:
      raise Exception(f"Cannot get abstract of {url}")

    # Get paragraph
    paragraph_tags_xpath = '//*[@id="detail_page"]/div[1]/div[2]/div/div[2]/div[4]'
    try:
      paragraph_tags = driver.find_elements(
          By.XPATH, paragraph_tags_xpath
      )
    except:
      raise Exception(f"Cannot get paragraphs of {url}")
    paragraphs = ' '.join([paragraph_tag.text.strip() for paragraph_tag in paragraph_tags])
    content = '\n\n'.join([title, author, abstract, paragraphs])
    # Save content to file
    article_name = f"article_{topic}_{article_id:05d}"
    article_save_path = os.path.join(
        root_dir, article_name
    )
    article_id += 1
    with open(article_save_path, "w", encoding="utf-8") as f:
      f.write(content)

100%|██████████| 7/7 [05:20<00:00, 45.76s/it]
